In [8]:
import os
import google.generativeai as genai
import random
import unicodedata
import time
import re
import sys

from google.colab import userdata


# Insira sua chave da API aqui ENTRE AS ASPAS
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

api_key = os.environ.get('GOOGLE_API_KEY')

if not api_key:
    print("Erro: A variável de ambiente GOOGLE_API_KEY não está definida.")
    sys.exit()

try:
    genai.configure(api_key=api_key)
except Exception as e:
    print(f"Erro ao configurar a biblioteca genai com a API Key: {e}")
    sys.exit()

try:
    model = genai.GenerativeModel('models/gemini-2.0-flash')
except Exception as e:
    print(f"Erro ao inicializar o modelo Gemini: {e}")
    sys.exit()

def remover_acentos(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

def agente_curadoria(nivel_dificuldade):
    prompt = f"""
    Você é um professor muito bem humorado e divertido que adora crianças e vai criar perguntas de ciências para crianças de 6 a 11 anos.
    Gere UMA ÚNICA pergunta de ciências de nível '{nivel_dificuldade}'.
    Para o nível inicial, pense em perguntas sobre fenômenos simples e observáveis, sempre com alternativas (A, B, C...) formatadas em linhas separadas.
    Para o nível médio, introduza conceitos um pouco mais abstratos, mas ainda relacionados ao cotidiano, também sempre com alternativas (A, B, C...) em linhas separadas.
    Para o nível avançado, pode abordar princípios científicos com mais detalhes, também com alternativas (A, B, C...) em linhas separadas.
    Tente adicionar um toque de humor leve, talvez uma pequena observação engraçada relacionada ao tema, sem confundir a pergunta.
    Inclua claramente a resposta correta, uma dica curta e uma explicação mais detalhada em sua resposta.

    Formate sua resposta da seguinte maneira EXATA, garantindo que a seção 'Resposta:' venha depois da pergunta e das alternativas:
    Pergunta: [a pergunta de ciências aqui]
    A) [Primeira alternativa]
    B) [Segunda alternativa]
    C) [Terceira alternativa]
    D) [Quarta alternativa]
    Resposta: [a letra da alternativa correta] - [uma dica curta sobre a resposta] - [a explicação completa da resposta correta]
    """

    try:
        response = model.generate_content(prompt)
        if response.text:
            pergunta_match = re.search(r"Pergunta:\s*(.*?)(?=\n[A-Z]\)|\nResposta:|$)", response.text, re.DOTALL | re.IGNORECASE)
            alternativas = re.findall(r"^([A-D])\)\s*(.*?)(?:\n|$)", response.text, re.MULTILINE | re.IGNORECASE)
            resposta_match = re.search(r"Resposta:\s*([A-D])(?: - (.*?))?(?: - (.*?))?$", response.text, re.DOTALL | re.IGNORECASE)

            if pergunta_match and alternativas and resposta_match:
                pergunta_raw = pergunta_match.group(1).strip()
                alternativa_correta_raw = resposta_match.group(1).strip()
                dica_curta = resposta_match.group(2).strip() if resposta_match.group(2) else "Pense um pouco mais!"
                explicacao_completa = resposta_match.group(3).strip() if resposta_match.group(3) else "A explicação está na resposta correta."

                pergunta_formatada = f"Pergunta: {pergunta_raw}\n" + "\n".join([f"{alt[0]}) {alt[1].strip()}" for alt in alternativas])
                alternativa_correta = alternativa_correta_raw.lower()

                return pergunta_formatada, alternativa_correta, dica_curta, explicacao_completa, nivel_dificuldade, False
            else:
                print(f"Formato inesperado na resposta do Gemini. Texto completo:\n---\n{response.text}\n---")
                return "Erro ao extrair informações da resposta do Gemini. Formato inesperado.", "", "", "", nivel_dificuldade, True
        else:
            print("Resposta vazia do Gemini.")
            return "Resposta vazia do Gemini.", "", "", "", nivel_dificuldade, True

    except Exception as e:
        print(f"Exceção durante a geração de conteúdo com Gemini: {e}")
        return f"Erro ao comunicar com o Gemini: {e}", "", "", "", nivel_dificuldade, True

def agente_apresentacao(pergunta, nivel):
    mensagens_divertidas = [
        "Prepare sua mente curiosa! Lá vem uma pergunta...",
        "Atenção, cientista em treinamento! Que comece a investigação...",
        "Se prepare para desvendar os mistérios da ciência! Eis a questão...",
        "Com vocês, diretamente do laboratório do saber...",
        "Quebra-cabeças científico chegando em 3, 2, 1...",
    ]
    mensagem_aleatoria = random.choice(mensagens_divertidas)
    print(f"\n{mensagem_aleatoria} (Nível: {nivel})\n")
    print(f"{pergunta}")

def agente_avaliacao(resposta_usuario, resposta_correta):
    resposta_usuario = remover_acentos(resposta_usuario).strip().lower()
    resposta_correta = remover_acentos(resposta_correta).strip().lower()

    if len(resposta_usuario) > 1 and resposta_usuario.endswith(')'):
        resposta_usuario = resposta_usuario[:-1]

    if resposta_usuario == resposta_correta:
        feedback_positivo = random.choice([
            "Você acertou em cheio! Que mente brilhante!",
            "Resposta correta! Você é um verdadeiro gênio da ciência!",
            "Isso mesmo! Você desvendou o mistério!",
            "Excelente! Seu conhecimento científico é incrível!",
            "Bingo! Você mandou muito bem!",
        ])
        return feedback_positivo, True
    else:
        feedback_negativo = random.choice([
            "Quase lá! Mas não foi dessa vez.",
            "Hmm, não exatamente.",
            "Essa não é a resposta correta.",
            "Oh, quase!",
            "Não se preocupe! Errar faz parte do aprendizado.",
        ])
        return feedback_negativo, False

print("Bem-vindo ao Quiz de Ciências Divertido!")
nivel_escolhido = input("Escolha o nível de dificuldade (inicial/medio/avancado): ").lower()
nivel_escolhido = remover_acentos(nivel_escolhido)
while nivel_escolhido not in ['inicial', 'medio', 'avancado']:
    nivel_escolhido = input("Nível inválido. Escolha entre inicial, medio ou avancado: ").lower()
    nivel_escolhido = remover_acentos(nivel_escolhido)

tentativas_geracao = 0
max_tentativas_geracao = 5

while True:
    pergunta, resposta_correta, dica_curta, explicacao_completa, nivel_pergunta, erro_na_geracao = agente_curadoria(nivel_escolhido)
    tentativas_geracao += 1

    if erro_na_geracao:
        print(f"Ops! Parece que o professor Gemini está pensando um pouco demais (tentativa {tentativas_geracao}/{max_tentativas_geracao}). Vamos tentar outra vez...")
        time.sleep(2)
        if tentativas_geracao >= max_tentativas_geracao:
            print("Desculpe, estou tendo dificuldades em gerar perguntas no momento. Verifique sua API Key e tente novamente mais tarde ou mude o nível.")
            break
        continue

    tentativas_geracao = 0
    agente_apresentacao(pergunta, nivel_pergunta)

    resposta_usuario = input("Sua resposta (apenas a letra da alternativa): ").strip().lower()
    while not re.match(r"^[a-d]\)?$", remover_acentos(resposta_usuario).strip().lower()):
        resposta_usuario = input("Resposta inválida. Por favor, digite apenas a letra da alternativa (ex: a, b, c, d): ").strip().lower()

    feedback, acertou = agente_avaliacao(resposta_usuario, resposta_correta)
    print(f"\nFeedback: {feedback}")

    if not acertou:
        tentar_novamente = input("Quer tentar novamente com uma dica? (sim/não): ").lower()
        if tentar_novamente == 'sim':
            print(f"\nDica: {dica_curta}")
            resposta_usuario_segunda_tentativa = input("Sua segunda tentativa (apenas a letra): ").strip().lower()
            while not re.match(r"^[a-d]\)?$", remover_acentos(resposta_usuario_segunda_tentativa).strip().lower()):
                resposta_usuario_segunda_tentativa = input("Resposta inválida. Por favor, digite apenas a letra da alternativa (ex: a, b, c, d): ").strip().lower()
            feedback_segunda_tentativa, acertou_segunda_tentativa = agente_avaliacao(resposta_usuario_segunda_tentativa, resposta_correta)
            print(f"\nFeedback da segunda tentativa: {feedback_segunda_tentativa}")
            if not acertou_segunda_tentativa:
                print(f"A resposta correta era: {resposta_correta.upper()} - {explicacao_completa}")
        else:
            print(f"A resposta correta era: {resposta_correta.upper()} - {explicacao_completa}")

    print("\nO que você quer fazer agora?")
    while True:
        opcao = input("[1] Continuar no mesmo nível\n[2] Mudar de nível\n[3] Parar o jogo\nDigite o número da sua opção: ")
        if opcao in ['1', '2', '3']:
            break
        else:
            print("Opção inválida. Por favor, digite 1, 2 ou 3.")

    if opcao == '2':
        nivel_anterior = nivel_escolhido
        nivel_escolhido = input("Escolha o novo nível de dificuldade (inicial/medio/avancado): ").lower()
        nivel_escolhido = remover_acentos(nivel_escolhido)
        while nivel_escolhido not in ['inicial', 'medio', 'avancado']:
            nivel_escolhido = input("Nível inválido. Escolha entre inicial, medio ou avancado: ").lower()
            nivel_escolhido = remover_acentos(nivel_escolhido)
        print(f"Ok! Mudando do nível '{nivel_anterior}' para '{nivel_escolhido}'.")
        tentativas_geracao = 0
    elif opcao == '3':
        break
    elif opcao == '1':
        print(f"Ok! Continuando no nível '{nivel_escolhido}'.")

print("\nObrigado por jogar! Até a próxima aventura científica!")

Bem-vindo ao Quiz de Ciências Divertido!


KeyboardInterrupt: Interrupted by user